In [ ]:
import os
import sys

# The path of the current Python script.
_CURRENT_PATH = os.path.dirname(os.path.realpath(__file__))
_TOP_PATH     = os.path.join(_CURRENT_PATH, 'dsta_mvs')

if _TOP_PATH not in sys.path:
    sys.path.insert( 0, _TOP_PATH)
    for i, p in enumerate(sys.path):
        print(f'{i}: {p}')

from api.dataset_player import DatasetProxy

dataset_player = DatasetProxy("./config.py", "./images")

print(dataset_player.dataset.frame_graph.g.nodes)
camera_frame = dataset_player.dataset.map_camera_frame
cameras = {k: v for k, v in camera_frame.items() if 'cam' in k}
print(cameras)
transformations = dict()
for i in cameras.keys():
    for j in cameras.keys():
        if i != j:
            matrix = dataset_player.dataset.frame_graph.query_transform(f0=cameras[i], f1=cameras[j])
            transformations[f"{i}-{j}"] = matrix.tolist()

In [ ]:
import requests
response = requests.request("GET", "http://127.0.0.1:9233/transformations")
D = response.json()['transformations']
D